In [2]:
from thinkbayes2 import Pmf

# Chapter 2
## PMF Class
Page 11

In [82]:
#----------------------------------------page 11
pmf = Pmf()
for x in [1,2,3,4,5,6]:
    pmf.Set(x, 1/6.0)

In [83]:
pmf

Pmf({1: 0.16666666666666666, 2: 0.16666666666666666, 3: 0.16666666666666666, 4: 0.16666666666666666, 5: 0.16666666666666666, 6: 0.16666666666666666})

In [84]:
word_list = ['a', 'rachel', 'house']
pmf = Pmf()
for word in word_list:
    pmf.Incr(word, 1)

In [85]:
pmf.Normalize()

3

In [86]:
pmf

Pmf({'a': 0.3333333333333333, 'rachel': 0.3333333333333333, 'house': 0.3333333333333333})

In [87]:
pmf.Prob('a')

0.3333333333333333

In [88]:
print(pmf.Prob('a'))

0.3333333333333333


### The Cookie Problem
p.12

In [12]:
# make the priors
pmf = Pmf()
pmf.Set('Bowl 1', 0.5)
pmf.Set('Bowl 2', 0.5)

In [13]:
# update the distribution based on the new data (the vanilla cookie):
#   multiply the priors by corresponding likelihoods of 
#   drawing a vanilla cookie from each bowl
#   to get numerator of Baye's Theorem
#     P(H)*P(D|H)
pmf.Mult('Bowl 1', 0.75)
pmf.Mult('Bowl 2', 0.5)

In [14]:
# normalize to get the posterior distribution P(H|D)
#   because the hypothesis are mutually exclusive and exhaustive,
#   we can re-normalize
pmf.Normalize()

0.625

In [18]:
# get posterior probability for bowl 1
#pmf.Prob('Bowl 1')
print(pmf.Prob('Bowl 1'))

0.6000000000000001


In [21]:
# I was expecting to see 3/5...checks out
3/5

0.6

### Bayesian Framework
p.13

In [45]:
class Cookie(Pmf):
    mixes = {'Bowl 1' :dict(vanilla=0.75, chocolate=0.25),
         'Bowl 2' :dict(vanilla=0.5, chocolate=0.5)
        }
    def __init__(self, hypos):
        Pmf.__init__(self)
        for hypo in hypos:
            self.Set(hypo, 1)
        self.Normalize()
    def Update(self, data):
        for hypo in self.Values():
            like = self.Likelihood(data, hypo)
            self.Mult(hypo, like)
        self.Normalize()
    def Likelihood(self, data, hypo):
        mix = self.mixes[hypo]
        like = mix[data]
        return like

In [46]:
hypos = ['Bowl 1', 'Bowl 2']
pmf = Cookie(hypos)

In [47]:
pmf.Update('vanilla')

In [48]:
for hypo, prob, in pmf.Items():
    print(hypo, prob)

Bowl 1 0.6000000000000001
Bowl 2 0.4


In [50]:
# example where we draw more than 1 cookie
dataset = ['vanilla', 'chocolate', 'vanilla', 'vanilla']
for data in dataset:
    pmf.Update(data)
for hypo, prob, in pmf.Items():
    print(hypo, prob)

Bowl 1 0.7401015228426396
Bowl 2 0.2598984771573604


## Monty Hall Problem
p.15

In [61]:
class Monty(Pmf):
    def __init__(self, hypos):
        Pmf.__init__(self)
        for hypo in hypos:
            self.Set(hypo, 1)
        self.Normalize()
    def Update(self, data):
        for hypo in self.Values():
            like = self.Likelihood(data, hypo)
            self.Mult(hypo, like)
        self.Normalize()
    def Likelihood(self, data, hypo):
        if hypo == data:
            return 0
        elif hypo == 'A':
            return 0.5
        else:
            return 1
    
        

In [62]:
hypos = 'ABC'
pmf = Monty(hypos)

In [63]:
# print BEFORE Monty opened the door
for hypo, prob in pmf.Items():
    print(hypo, prob)

A 0.3333333333333333
B 0.3333333333333333
C 0.3333333333333333


In [64]:
data = 'B'
pmf.Update(data)

In [65]:
# print AFTER Monty opened the door
for hypo, prob in pmf.Items():
    print(hypo, prob)

A 0.3333333333333333
B 0.0
C 0.6666666666666666


## Encapsulation and M&M problem
p.16

In [66]:
from thinkbayes2 import Suite

In [81]:
class MM(Suite):
    hypotheses = dict(A=hypoA, B=hypoB)
    def Likelihood(self, data, hypo):
        bag, color = data
        mix = self.hypotheses[hypo][bag]
        like = mix[color]
        return like

In [82]:
mix94 = dict(brown=30,
            yellow=20,
            red=20,
            green=10,
            orange=10,
            tan=10)
mix96 = dict(blue=24,
            green=20,
            orange=16,
            yellow=14,
            red=13,
            brown=13)

In [83]:
hypoA = dict(bag1=mix94, bag2=mix96)
hypoB = dict(bag1=mix96, bag2=mix94)

In [84]:
suite = MM('AB')

In [85]:
suite.Update(('bag1', 'yellow'))

17.0

In [86]:
suite.Update(('bag2', 'green'))

15.882352941176471

In [87]:
suite.Print()

A 0.7407407407407407
B 0.2592592592592592


In [88]:
20/27

0.7407407407407407